# Lab 2: EOG Signal Analysis and Gaze Classification

**BioRobotics - Introduction to Electrooculography**

This notebook guides you through the analysis of EOG (electrooculography) data collected with the BioRadio device. You will:

1. **Load and visualize** raw EOG signals
2. **Process signals** with baseline correction and filtering
3. **Detect blink artifacts** in the data
4. **Extract features** for classification
5. **Apply dimensionality reduction** using PCA and ICA
6. **Classify gaze directions** using SVM (Linear and RBF kernels)
7. **Evaluate performance** with confusion matrices and metrics

---

## Setup and Imports

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import glob
import re
import warnings
warnings.filterwarnings('ignore')

# Signal processing
from scipy import signal
from scipy.ndimage import uniform_filter1d

# Machine learning
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.decomposition import PCA, FastICA
from sklearn.svm import SVC
from sklearn.metrics import (
    confusion_matrix, classification_report, accuracy_score,
    ConfusionMatrixDisplay
)

# Plotting settings
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("All libraries imported successfully!")

## Configuration

Set your data directory and recording parameters below:

In [ ]:
# === CONFIGURATION - EDIT THESE ===

# Path to your recorded data folder
DATA_DIR = Path("../data")  # Adjust this to your data location

# Recording parameters
SAMPLE_RATE = 250  # Hz (should match your BioRadio configuration)

# Channel mapping
HEOG_CHANNEL = 0  # Horizontal EOG (Channel 1 on BioRadio)
VEOG_CHANNEL = 1  # Vertical EOG (Channel 2 on BioRadio)

# Gaze classes to analyze
GAZE_CLASSES = ['center', 'left', 'right', 'up', 'down', 'blink', 'double_blink']

# Filter parameters
LOWPASS_CUTOFF = 30  # Hz - EOG content is below 30 Hz
HIGHPASS_CUTOFF = 0.1  # Hz - removes very slow drift (optional)

print(f"Data directory: {DATA_DIR.absolute()}")
print(f"Sample rate: {SAMPLE_RATE} Hz")
print(f"Gaze classes: {GAZE_CLASSES}")

---

## Part 1: Loading and Exploring the Data

First, let's discover what data files are available and load them into a structured format.

In [ ]:
def find_data_files(data_dir, pattern="*.csv"):
    """
    Find all CSV files in the data directory and extract metadata from filenames.
    
    Expected filename format: {participant}_{gaze}_{trial}_{timestamp}.csv
    Example: P01_left_trial003_20250206_143022.csv
    """
    data_dir = Path(data_dir)
    files = []
    
    # Search recursively
    for csv_file in data_dir.rglob(pattern):
        filename = csv_file.stem
        
        # Try to parse the filename
        # Pattern: participant_gaze_trial###_timestamp
        parts = filename.split('_')
        
        if len(parts) >= 3:
            participant = parts[0]
            gaze = parts[1]
            
            # Extract trial number
            trial_match = re.search(r'trial(\d+)', filename, re.IGNORECASE)
            trial = int(trial_match.group(1)) if trial_match else 0
            
            files.append({
                'path': csv_file,
                'participant': participant,
                'gaze': gaze.lower(),
                'trial': trial,
                'filename': filename
            })
    
    return pd.DataFrame(files)

# Find all data files
file_df = find_data_files(DATA_DIR)

if len(file_df) == 0:
    print("⚠️ No data files found!")
    print(f"   Searched in: {DATA_DIR.absolute()}")
    print("   Make sure your data files are in the correct location.")
else:
    print(f"Found {len(file_df)} data files\n")
    
    # Summary by gaze class
    print("Trials per gaze class:")
    print(file_df.groupby('gaze').size().to_string())
    print()
    
    # Summary by participant
    print("\nTrials per participant:")
    print(file_df.groupby('participant').size().to_string())

In [ ]:
def load_eog_file(filepath):
    """
    Load a single EOG data file.
    
    Returns:
        DataFrame with columns: timestamp, heog, veog (and any other channels)
    """
    df = pd.read_csv(filepath)
    
    # Standardize column names
    df.columns = df.columns.str.lower().str.strip()
    
    # If columns are numbered (ch0, ch1, etc.), rename them
    if 'ch0' in df.columns or 'channel_0' in df.columns:
        rename_map = {
            'ch0': 'heog', 'channel_0': 'heog',
            'ch1': 'veog', 'channel_1': 'veog',
        }
        df = df.rename(columns=rename_map)
    
    # If no timestamp column, create one based on sample rate
    if 'timestamp' not in df.columns and 'time' not in df.columns:
        df['timestamp'] = np.arange(len(df)) / SAMPLE_RATE
    elif 'time' in df.columns:
        df = df.rename(columns={'time': 'timestamp'})
    
    return df

# Load a sample file to inspect the structure
if len(file_df) > 0:
    sample_file = file_df.iloc[0]['path']
    sample_data = load_eog_file(sample_file)
    
    print(f"Sample file: {sample_file.name}")
    print(f"Shape: {sample_data.shape}")
    print(f"Columns: {list(sample_data.columns)}")
    print(f"Duration: {len(sample_data) / SAMPLE_RATE:.2f} seconds")
    print("\nFirst few rows:")
    display(sample_data.head())

In [ ]:
def load_all_trials(file_df, gaze_classes=None):
    """
    Load all trials into a structured dictionary.
    
    Returns:
        dict: {gaze_class: [list of DataFrames]}
    """
    if gaze_classes is None:
        gaze_classes = file_df['gaze'].unique()
    
    all_data = {gaze: [] for gaze in gaze_classes}
    
    for _, row in file_df.iterrows():
        gaze = row['gaze']
        if gaze in gaze_classes:
            try:
                df = load_eog_file(row['path'])
                df['participant'] = row['participant']
                df['trial'] = row['trial']
                df['gaze'] = gaze
                all_data[gaze].append(df)
            except Exception as e:
                print(f"Error loading {row['path']}: {e}")
    
    # Print summary
    print("Loaded trials per class:")
    for gaze, trials in all_data.items():
        if trials:
            total_samples = sum(len(t) for t in trials)
            print(f"  {gaze}: {len(trials)} trials, {total_samples} samples")
    
    return all_data

# Load all data
if len(file_df) > 0:
    all_data = load_all_trials(file_df, GAZE_CLASSES)
else:
    print("No data to load. Using synthetic data for demonstration...")
    all_data = None

### Visualize Raw EOG Signals

Let's look at example signals for each gaze direction:

In [ ]:
def plot_raw_eog(data_dict, num_examples=1):
    """
    Plot raw EOG signals for each gaze class.
    """
    gaze_classes = [g for g in data_dict.keys() if data_dict[g]]
    n_classes = len(gaze_classes)
    
    if n_classes == 0:
        print("No data to plot")
        return
    
    fig, axes = plt.subplots(n_classes, 2, figsize=(14, 3 * n_classes))
    if n_classes == 1:
        axes = axes.reshape(1, -1)
    
    for i, gaze in enumerate(gaze_classes):
        trials = data_dict[gaze]
        if not trials:
            continue
            
        # Plot first trial
        trial = trials[0]
        time = np.arange(len(trial)) / SAMPLE_RATE
        
        # HEOG
        ax1 = axes[i, 0]
        if 'heog' in trial.columns:
            ax1.plot(time, trial['heog'], 'b-', linewidth=0.8)
        ax1.set_ylabel('HEOG (μV)')
        ax1.set_title(f'{gaze.upper()} - Horizontal EOG')
        ax1.set_xlabel('Time (s)')
        
        # VEOG
        ax2 = axes[i, 1]
        if 'veog' in trial.columns:
            ax2.plot(time, trial['veog'], 'r-', linewidth=0.8)
        ax2.set_ylabel('VEOG (μV)')
        ax2.set_title(f'{gaze.upper()} - Vertical EOG')
        ax2.set_xlabel('Time (s)')
    
    plt.tight_layout()
    plt.show()

if all_data:
    plot_raw_eog(all_data)

---

## Question 1 Response

**Sketch the EOG waveforms you observe for each gaze direction. Which channel responds to which movement?**

*Your observations:*

- **Left gaze:** HEOG shows ___ deflection, VEOG shows ___
- **Right gaze:** HEOG shows ___ deflection, VEOG shows ___
- **Up gaze:** HEOG shows ___, VEOG shows ___ deflection
- **Down gaze:** HEOG shows ___, VEOG shows ___ deflection
- **Center:** Both channels show ___
- **Approximate amplitude:** ___ μV per gaze direction

---

## Part 2: Signal Processing

EOG signals require specific processing to remove artifacts and prepare for analysis.

In [ ]:
def baseline_correction(signal_data, method='mean'):
    """
    Remove DC offset from the signal.
    
    Args:
        signal_data: 1D array of signal values
        method: 'mean' (subtract mean) or 'median' (subtract median)
        
    Returns:
        Baseline-corrected signal
    """
    if method == 'mean':
        return signal_data - np.mean(signal_data)
    elif method == 'median':
        return signal_data - np.median(signal_data)
    else:
        raise ValueError(f"Unknown method: {method}")


def remove_drift(signal_data, window_size=None, sample_rate=SAMPLE_RATE):
    """
    Remove slow baseline drift using a moving average subtraction.
    
    Args:
        signal_data: 1D array of signal values
        window_size: Size of moving average window in samples
                     Default is 2 seconds worth of samples
    
    Returns:
        Drift-corrected signal
    """
    if window_size is None:
        window_size = int(2 * sample_rate)  # 2 second window
    
    # Compute moving average (baseline estimate)
    baseline = uniform_filter1d(signal_data, size=window_size, mode='nearest')
    
    return signal_data - baseline


def lowpass_filter(signal_data, cutoff=LOWPASS_CUTOFF, sample_rate=SAMPLE_RATE, order=4):
    """
    Apply a low-pass Butterworth filter.
    
    Args:
        signal_data: 1D array of signal values
        cutoff: Cutoff frequency in Hz
        sample_rate: Sampling rate in Hz
        order: Filter order
        
    Returns:
        Filtered signal
    """
    nyquist = sample_rate / 2
    normalized_cutoff = cutoff / nyquist
    
    b, a = signal.butter(order, normalized_cutoff, btype='low')
    
    # Use filtfilt for zero-phase filtering
    return signal.filtfilt(b, a, signal_data)


def bandpass_filter(signal_data, low_cutoff=HIGHPASS_CUTOFF, high_cutoff=LOWPASS_CUTOFF,
                    sample_rate=SAMPLE_RATE, order=4):
    """
    Apply a band-pass Butterworth filter.
    
    Args:
        signal_data: 1D array of signal values
        low_cutoff: Low cutoff frequency in Hz
        high_cutoff: High cutoff frequency in Hz
        
    Returns:
        Filtered signal
    """
    nyquist = sample_rate / 2
    low = low_cutoff / nyquist
    high = high_cutoff / nyquist
    
    b, a = signal.butter(order, [low, high], btype='band')
    
    return signal.filtfilt(b, a, signal_data)


def process_eog_signal(signal_data, sample_rate=SAMPLE_RATE,
                       remove_baseline=True, remove_slow_drift=False,
                       apply_lowpass=True, lowpass_cutoff=LOWPASS_CUTOFF):
    """
    Full processing pipeline for EOG signals.
    
    Args:
        signal_data: 1D array of signal values
        sample_rate: Sampling rate in Hz
        remove_baseline: Subtract mean from signal
        remove_slow_drift: Apply drift removal (moving average subtraction)
        apply_lowpass: Apply low-pass filter
        lowpass_cutoff: Cutoff frequency for low-pass filter
        
    Returns:
        Processed signal
    """
    processed = signal_data.copy()
    
    # Step 1: Baseline correction
    if remove_baseline:
        processed = baseline_correction(processed, method='mean')
    
    # Step 2: Remove slow drift (optional)
    if remove_slow_drift:
        processed = remove_drift(processed, sample_rate=sample_rate)
    
    # Step 3: Low-pass filter
    if apply_lowpass:
        processed = lowpass_filter(processed, cutoff=lowpass_cutoff,
                                   sample_rate=sample_rate)
    
    return processed

print("Signal processing functions defined!")

### Compare Raw vs. Processed Signals

In [ ]:
def plot_processing_comparison(trial_data, sample_rate=SAMPLE_RATE):
    """
    Show the effect of signal processing on a single trial.
    """
    fig, axes = plt.subplots(2, 2, figsize=(14, 8))
    time = np.arange(len(trial_data)) / sample_rate
    
    channels = [('heog', 'HEOG', 'blue'), ('veog', 'VEOG', 'red')]
    
    for col, (ch_name, ch_label, color) in enumerate(channels):
        if ch_name not in trial_data.columns:
            continue
            
        raw = trial_data[ch_name].values
        processed = process_eog_signal(raw, sample_rate=sample_rate)
        
        # Raw signal
        axes[0, col].plot(time, raw, color=color, linewidth=0.8, alpha=0.8)
        axes[0, col].set_title(f'{ch_label} - Raw Signal')
        axes[0, col].set_xlabel('Time (s)')
        axes[0, col].set_ylabel('Amplitude (μV)')
        
        # Processed signal
        axes[1, col].plot(time, processed, color=color, linewidth=0.8, alpha=0.8)
        axes[1, col].set_title(f'{ch_label} - Processed (baseline corrected + low-pass filtered)')
        axes[1, col].set_xlabel('Time (s)')
        axes[1, col].set_ylabel('Amplitude (μV)')
    
    plt.tight_layout()
    plt.show()

# Plot comparison for one trial
if all_data:
    # Find a gaze class with data
    for gaze in ['left', 'right', 'center']:
        if gaze in all_data and all_data[gaze]:
            print(f"Processing example from '{gaze}' class:")
            plot_processing_comparison(all_data[gaze][0])
            break

---

## Question 5 Response

**Compare the raw and filtered EOG signals. What types of noise does the low-pass filter remove? How does baseline correction affect the signal?**

*Your observations:*

- The low-pass filter removes: ___
- Baseline correction affects the signal by: ___
- The processed signal is better for classification because: ___

---

## Part 3: Blink Detection

Blinks create large, characteristic artifacts in the VEOG channel. We can detect them for removal or use them as a control signal.

In [ ]:
def detect_blinks(veog_signal, sample_rate=SAMPLE_RATE,
                  threshold_factor=3.0, min_duration_ms=50, max_duration_ms=400):
    """
    Detect blinks in the VEOG signal using threshold crossing.
    
    Blinks appear as large, sharp positive deflections in VEOG.
    
    Args:
        veog_signal: 1D array of VEOG values
        sample_rate: Sampling rate in Hz
        threshold_factor: Multiple of std dev to use as threshold
        min_duration_ms: Minimum blink duration in milliseconds
        max_duration_ms: Maximum blink duration in milliseconds
        
    Returns:
        list of tuples: [(start_sample, end_sample, peak_sample), ...]
    """
    # Process signal
    processed = baseline_correction(veog_signal)
    
    # Calculate threshold
    std = np.std(processed)
    threshold = threshold_factor * std
    
    # Find samples above threshold
    above_threshold = processed > threshold
    
    # Find blink regions
    blinks = []
    min_samples = int(min_duration_ms * sample_rate / 1000)
    max_samples = int(max_duration_ms * sample_rate / 1000)
    
    in_blink = False
    blink_start = 0
    
    for i, above in enumerate(above_threshold):
        if above and not in_blink:
            # Start of potential blink
            in_blink = True
            blink_start = i
        elif not above and in_blink:
            # End of potential blink
            in_blink = False
            blink_end = i
            duration = blink_end - blink_start
            
            # Check duration criteria
            if min_samples <= duration <= max_samples:
                # Find peak within blink region
                peak_idx = blink_start + np.argmax(processed[blink_start:blink_end])
                blinks.append((blink_start, blink_end, peak_idx))
    
    return blinks


def plot_blink_detection(trial_data, sample_rate=SAMPLE_RATE):
    """
    Visualize blink detection on VEOG signal.
    """
    if 'veog' not in trial_data.columns:
        print("No VEOG channel found")
        return
    
    veog = trial_data['veog'].values
    time = np.arange(len(veog)) / sample_rate
    
    # Detect blinks
    blinks = detect_blinks(veog, sample_rate=sample_rate)
    
    # Plot
    fig, ax = plt.subplots(figsize=(14, 4))
    ax.plot(time, veog, 'b-', linewidth=0.8, label='VEOG')
    
    # Mark detected blinks
    for start, end, peak in blinks:
        ax.axvspan(start/sample_rate, end/sample_rate, 
                   color='red', alpha=0.3, label='Blink' if start == blinks[0][0] else '')
        ax.plot(peak/sample_rate, veog[peak], 'r^', markersize=10)
    
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('VEOG (μV)')
    ax.set_title(f'Blink Detection - Found {len(blinks)} blink(s)')
    ax.legend()
    plt.tight_layout()
    plt.show()
    
    return blinks

# Test blink detection on blink trials
if all_data and 'blink' in all_data and all_data['blink']:
    print("Blink detection on 'blink' class trial:")
    detected = plot_blink_detection(all_data['blink'][0])

---

## Question 2 Response

**How do blink artifacts appear in the HEOG vs. VEOG channels? Why might blinks primarily affect the VEOG channel?**

*Your observations:*

- In the VEOG channel, blinks appear as: ___
- In the HEOG channel, blinks appear as: ___
- Blinks primarily affect VEOG because: ___

---

## Part 4: Feature Extraction

Extract features from each trial for classification.

In [ ]:
def extract_features(trial_data, sample_rate=SAMPLE_RATE):
    """
    Extract features from a single EOG trial.
    
    Features extracted:
    - Mean amplitude (HEOG, VEOG)
    - Standard deviation (HEOG, VEOG)
    - Peak-to-peak amplitude (HEOG, VEOG)
    - RMS (Root Mean Square) (HEOG, VEOG)
    - Max absolute value (HEOG, VEOG)
    - Signal range (HEOG, VEOG)
    - Zero crossings (HEOG, VEOG)
    - Energy (sum of squared values)
    
    Returns:
        dict of features
    """
    features = {}
    
    for ch_name in ['heog', 'veog']:
        if ch_name not in trial_data.columns:
            continue
        
        # Get raw and processed signals
        raw = trial_data[ch_name].values
        processed = process_eog_signal(raw, sample_rate=sample_rate)
        
        prefix = ch_name.upper()
        
        # Time-domain features on processed signal
        features[f'{prefix}_mean'] = np.mean(processed)
        features[f'{prefix}_std'] = np.std(processed)
        features[f'{prefix}_min'] = np.min(processed)
        features[f'{prefix}_max'] = np.max(processed)
        features[f'{prefix}_ptp'] = np.ptp(processed)  # peak-to-peak
        features[f'{prefix}_rms'] = np.sqrt(np.mean(processed**2))
        features[f'{prefix}_abs_max'] = np.max(np.abs(processed))
        features[f'{prefix}_energy'] = np.sum(processed**2)
        
        # Zero crossings (indicates oscillation)
        zero_crossings = np.sum(np.diff(np.sign(processed)) != 0)
        features[f'{prefix}_zero_crossings'] = zero_crossings
        
        # Skewness and kurtosis
        from scipy.stats import skew, kurtosis
        features[f'{prefix}_skew'] = skew(processed)
        features[f'{prefix}_kurtosis'] = kurtosis(processed)
        
        # Derivative features (velocity)
        velocity = np.diff(processed) * sample_rate
        features[f'{prefix}_velocity_max'] = np.max(np.abs(velocity))
        features[f'{prefix}_velocity_std'] = np.std(velocity)
    
    # Combined features
    if 'heog' in trial_data.columns and 'veog' in trial_data.columns:
        heog = process_eog_signal(trial_data['heog'].values)
        veog = process_eog_signal(trial_data['veog'].values)
        
        # Ratio features
        heog_energy = np.sum(heog**2)
        veog_energy = np.sum(veog**2)
        total_energy = heog_energy + veog_energy
        
        if total_energy > 0:
            features['HEOG_energy_ratio'] = heog_energy / total_energy
            features['VEOG_energy_ratio'] = veog_energy / total_energy
        
        # Correlation between channels
        features['channel_correlation'] = np.corrcoef(heog, veog)[0, 1]
    
    return features


def extract_all_features(data_dict, sample_rate=SAMPLE_RATE):
    """
    Extract features from all trials in the dataset.
    
    Returns:
        DataFrame with features and labels
    """
    all_features = []
    
    for gaze_class, trials in data_dict.items():
        for i, trial in enumerate(trials):
            try:
                features = extract_features(trial, sample_rate=sample_rate)
                features['gaze'] = gaze_class
                features['trial'] = i
                if 'participant' in trial.columns:
                    features['participant'] = trial['participant'].iloc[0]
                all_features.append(features)
            except Exception as e:
                print(f"Error extracting features from {gaze_class} trial {i}: {e}")
    
    return pd.DataFrame(all_features)

# Extract features from all data
if all_data:
    feature_df = extract_all_features(all_data)
    print(f"Extracted features from {len(feature_df)} trials")
    print(f"Number of features: {len([c for c in feature_df.columns if c not in ['gaze', 'trial', 'participant']])}")
    print("\nFeature summary:")
    display(feature_df.describe())

### Visualize Feature Distributions

In [ ]:
def plot_feature_distributions(feature_df, features_to_plot=None):
    """
    Plot distributions of key features by gaze class.
    """
    if features_to_plot is None:
        features_to_plot = ['HEOG_mean', 'VEOG_mean', 'HEOG_std', 'VEOG_std']
    
    # Filter to features that exist
    features_to_plot = [f for f in features_to_plot if f in feature_df.columns]
    
    if not features_to_plot:
        print("No features to plot")
        return
    
    n_features = len(features_to_plot)
    fig, axes = plt.subplots(1, n_features, figsize=(4 * n_features, 4))
    if n_features == 1:
        axes = [axes]
    
    gaze_classes = feature_df['gaze'].unique()
    colors = plt.cm.tab10(np.linspace(0, 1, len(gaze_classes)))
    
    for ax, feature in zip(axes, features_to_plot):
        for gaze, color in zip(gaze_classes, colors):
            data = feature_df[feature_df['gaze'] == gaze][feature]
            ax.hist(data, bins=15, alpha=0.5, label=gaze, color=color)
        
        ax.set_xlabel(feature)
        ax.set_ylabel('Count')
        ax.legend()
    
    plt.tight_layout()
    plt.show()

if all_data and 'feature_df' in dir():
    plot_feature_distributions(feature_df)

In [ ]:
def plot_feature_scatter(feature_df, x_feature='HEOG_mean', y_feature='VEOG_mean'):
    """
    Create a scatter plot of two features colored by gaze class.
    """
    if x_feature not in feature_df.columns or y_feature not in feature_df.columns:
        print(f"Features not found. Available: {list(feature_df.columns)}")
        return
    
    fig, ax = plt.subplots(figsize=(10, 8))
    
    gaze_classes = feature_df['gaze'].unique()
    colors = plt.cm.tab10(np.linspace(0, 1, len(gaze_classes)))
    
    for gaze, color in zip(gaze_classes, colors):
        mask = feature_df['gaze'] == gaze
        ax.scatter(feature_df[mask][x_feature], 
                   feature_df[mask][y_feature],
                   c=[color], label=gaze, s=60, alpha=0.7, edgecolors='white')
    
    ax.set_xlabel(x_feature)
    ax.set_ylabel(y_feature)
    ax.set_title(f'{x_feature} vs {y_feature} by Gaze Class')
    ax.legend()
    plt.tight_layout()
    plt.show()

if all_data and 'feature_df' in dir():
    plot_feature_scatter(feature_df, 'HEOG_mean', 'VEOG_mean')

---

## Question 6 Response

**Which features show the most separation between different gaze directions?**

*Your observations:*

- Features that separate horizontal gaze (left/right): ___
- Features that separate vertical gaze (up/down): ___
- Features that identify blinks: ___
- The most discriminative feature overall is: ___

---

## Part 5: Dimensionality Reduction

### 5.1 Principal Component Analysis (PCA)

In [ ]:
def apply_pca(feature_df, n_components=2):
    """
    Apply PCA to the feature matrix.
    
    Returns:
        pca: fitted PCA object
        X_pca: transformed data
        feature_names: list of feature names used
    """
    # Get numeric features only
    exclude_cols = ['gaze', 'trial', 'participant']
    feature_cols = [c for c in feature_df.columns if c not in exclude_cols]
    
    X = feature_df[feature_cols].values
    
    # Handle missing values
    X = np.nan_to_num(X, nan=0)
    
    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Apply PCA
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X_scaled)
    
    return pca, X_pca, feature_cols


def plot_pca_results(feature_df, pca, X_pca):
    """
    Visualize PCA results.
    """
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Scatter plot in PC space
    ax1 = axes[0]
    gaze_classes = feature_df['gaze'].unique()
    colors = plt.cm.tab10(np.linspace(0, 1, len(gaze_classes)))
    
    for gaze, color in zip(gaze_classes, colors):
        mask = feature_df['gaze'] == gaze
        ax1.scatter(X_pca[mask, 0], X_pca[mask, 1],
                    c=[color], label=gaze, s=60, alpha=0.7, edgecolors='white')
    
    ax1.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% variance)')
    ax1.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% variance)')
    ax1.set_title('PCA: Gaze Classes in PC Space')
    ax1.legend()
    
    # Explained variance
    ax2 = axes[1]
    cumulative_var = np.cumsum(pca.explained_variance_ratio_)
    ax2.bar(range(1, len(pca.explained_variance_ratio_) + 1),
            pca.explained_variance_ratio_, alpha=0.7, label='Individual')
    ax2.plot(range(1, len(cumulative_var) + 1), cumulative_var,
             'ro-', label='Cumulative')
    ax2.set_xlabel('Principal Component')
    ax2.set_ylabel('Explained Variance Ratio')
    ax2.set_title('PCA: Explained Variance')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()
    
    # Print variance explained
    print("\nVariance explained by each component:")
    for i, var in enumerate(pca.explained_variance_ratio_):
        print(f"  PC{i+1}: {var*100:.2f}%")
    print(f"  Total (first 2): {sum(pca.explained_variance_ratio_[:2])*100:.2f}%")

# Apply PCA
if all_data and 'feature_df' in dir() and len(feature_df) > 0:
    pca, X_pca, feature_names = apply_pca(feature_df, n_components=min(10, len(feature_df)))
    plot_pca_results(feature_df, pca, X_pca)

### 5.2 Independent Component Analysis (ICA)

In [ ]:
def apply_ica_to_signals(trial_data, n_components=2, sample_rate=SAMPLE_RATE):
    """
    Apply ICA directly to the raw EOG signals to separate sources.
    
    This can help separate eye movement signals from blink artifacts.
    
    Returns:
        ica: fitted ICA object
        S: independent components (source signals)
    """
    # Get HEOG and VEOG
    channels = []
    for ch in ['heog', 'veog']:
        if ch in trial_data.columns:
            processed = process_eog_signal(trial_data[ch].values, sample_rate=sample_rate)
            channels.append(processed)
    
    if len(channels) < 2:
        return None, None
    
    # Stack channels: shape (n_samples, n_channels)
    X = np.column_stack(channels)
    
    # Apply ICA
    ica = FastICA(n_components=n_components, random_state=42, max_iter=1000)
    S = ica.fit_transform(X)
    
    return ica, S


def plot_ica_comparison(trial_data, sample_rate=SAMPLE_RATE):
    """
    Compare original channels with ICA-separated components.
    """
    ica, S = apply_ica_to_signals(trial_data, sample_rate=sample_rate)
    
    if S is None:
        print("ICA failed - need at least 2 channels")
        return
    
    time = np.arange(len(trial_data)) / sample_rate
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 8))
    
    # Original channels
    axes[0, 0].plot(time, process_eog_signal(trial_data['heog'].values), 'b-', linewidth=0.8)
    axes[0, 0].set_title('Original HEOG')
    axes[0, 0].set_xlabel('Time (s)')
    axes[0, 0].set_ylabel('Amplitude')
    
    axes[0, 1].plot(time, process_eog_signal(trial_data['veog'].values), 'r-', linewidth=0.8)
    axes[0, 1].set_title('Original VEOG')
    axes[0, 1].set_xlabel('Time (s)')
    axes[0, 1].set_ylabel('Amplitude')
    
    # ICA components
    axes[1, 0].plot(time, S[:, 0], 'g-', linewidth=0.8)
    axes[1, 0].set_title('ICA Component 1')
    axes[1, 0].set_xlabel('Time (s)')
    axes[1, 0].set_ylabel('Amplitude')
    
    axes[1, 1].plot(time, S[:, 1], 'm-', linewidth=0.8)
    axes[1, 1].set_title('ICA Component 2')
    axes[1, 1].set_xlabel('Time (s)')
    axes[1, 1].set_ylabel('Amplitude')
    
    plt.tight_layout()
    plt.show()
    
    # Print mixing matrix
    print("\nICA Mixing Matrix (how components combine to form original signals):")
    print(f"  HEOG = {ica.mixing_[0, 0]:.3f} * IC1 + {ica.mixing_[0, 1]:.3f} * IC2")
    print(f"  VEOG = {ica.mixing_[1, 0]:.3f} * IC1 + {ica.mixing_[1, 1]:.3f} * IC2")

# Apply ICA to a trial with blinks
if all_data:
    # Try blink trials first, then any available trial
    for gaze in ['blink', 'double_blink', 'left', 'right', 'center']:
        if gaze in all_data and all_data[gaze]:
            print(f"ICA analysis on '{gaze}' trial:")
            plot_ica_comparison(all_data[gaze][0])
            break

---

## Question 7 Response

**How much variance is explained by each principal component? What do the first two principal components represent in terms of eye movements?**

*Your observations:*

- PC1 explains ___% of variance and seems to represent: ___
- PC2 explains ___% of variance and seems to represent: ___
- Total variance explained by first 2 PCs: ___

---

## Question 8 Response

**How do the ICA-separated components differ from the original HEOG and VEOG channels? Can you identify components that correspond to horizontal movement, vertical movement, and blinks?**

*Your observations:*

- ICA Component 1 appears to capture: ___
- ICA Component 2 appears to capture: ___
- The mixing matrix suggests: ___

---

## Part 6: Classification with SVM

Train and evaluate Support Vector Machine classifiers for gaze direction classification.

In [ ]:
def prepare_classification_data(feature_df, gaze_classes=None):
    """
    Prepare feature matrix and labels for classification.
    
    Returns:
        X: feature matrix (scaled)
        y: labels (encoded)
        feature_names: list of feature names
        label_encoder: fitted LabelEncoder
        scaler: fitted StandardScaler
    """
    # Filter to specified classes
    if gaze_classes is not None:
        df = feature_df[feature_df['gaze'].isin(gaze_classes)].copy()
    else:
        df = feature_df.copy()
    
    # Get feature columns
    exclude_cols = ['gaze', 'trial', 'participant']
    feature_cols = [c for c in df.columns if c not in exclude_cols]
    
    X = df[feature_cols].values
    X = np.nan_to_num(X, nan=0)  # Handle missing values
    
    # Encode labels
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(df['gaze'])
    
    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    return X_scaled, y, feature_cols, label_encoder, scaler


def train_and_evaluate_svm(X, y, label_encoder, kernel='rbf', test_size=0.3):
    """
    Train an SVM classifier and evaluate performance.
    
    Args:
        X: scaled feature matrix
        y: encoded labels
        label_encoder: for decoding labels
        kernel: 'linear' or 'rbf'
        test_size: fraction of data for testing
        
    Returns:
        model: trained SVM
        results: dict with accuracy, confusion matrix, etc.
    """
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=42, stratify=y
    )
    
    # Train SVM
    if kernel == 'linear':
        model = SVC(kernel='linear', C=1.0, random_state=42)
    else:  # rbf
        model = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
    
    model.fit(X_train, y_train)
    
    # Predict
    y_pred = model.predict(X_test)
    
    # Evaluate
    accuracy = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    report = classification_report(y_test, y_pred,
                                    target_names=label_encoder.classes_,
                                    output_dict=True)
    
    # Cross-validation
    cv_scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
    
    results = {
        'accuracy': accuracy,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'confusion_matrix': cm,
        'classification_report': report,
        'y_test': y_test,
        'y_pred': y_pred
    }
    
    return model, results

print("Classification functions defined!")

In [ ]:
def plot_confusion_matrix(cm, class_names, title='Confusion Matrix'):
    """
    Plot a confusion matrix with labels.
    """
    fig, ax = plt.subplots(figsize=(8, 6))
    
    im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    ax.figure.colorbar(im, ax=ax)
    
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           xticklabels=class_names,
           yticklabels=class_names,
           title=title,
           ylabel='True Label',
           xlabel='Predicted Label')
    
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")
    
    # Add text annotations
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], 'd'),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    
    plt.tight_layout()
    plt.show()


def compare_svm_kernels(feature_df, gaze_classes=None):
    """
    Train and compare Linear vs RBF SVM.
    """
    X, y, feature_names, label_encoder, scaler = prepare_classification_data(
        feature_df, gaze_classes
    )
    
    print(f"Dataset: {len(y)} samples, {len(np.unique(y))} classes")
    print(f"Features: {len(feature_names)}")
    print(f"Classes: {label_encoder.classes_}\n")
    
    results = {}
    
    for kernel in ['linear', 'rbf']:
        print(f"\n{'='*50}")
        print(f"  SVM with {kernel.upper()} kernel")
        print(f"{'='*50}")
        
        model, res = train_and_evaluate_svm(X, y, label_encoder, kernel=kernel)
        results[kernel] = res
        
        print(f"\n  Test Accuracy: {res['accuracy']*100:.2f}%")
        print(f"  Cross-validation: {res['cv_mean']*100:.2f}% (+/- {res['cv_std']*100:.2f}%)")
        
        print("\n  Classification Report:")
        for cls in label_encoder.classes_:
            metrics = res['classification_report'][cls]
            print(f"    {cls:15s} precision={metrics['precision']:.2f} "
                  f"recall={metrics['recall']:.2f} f1={metrics['f1-score']:.2f}")
        
        plot_confusion_matrix(res['confusion_matrix'], label_encoder.classes_,
                              title=f'Confusion Matrix - {kernel.upper()} SVM')
    
    # Summary comparison
    print("\n" + "="*50)
    print("  SUMMARY")
    print("="*50)
    print(f"  Linear SVM: {results['linear']['accuracy']*100:.2f}% "
          f"(CV: {results['linear']['cv_mean']*100:.2f}%)")
    print(f"  RBF SVM:    {results['rbf']['accuracy']*100:.2f}% "
          f"(CV: {results['rbf']['cv_mean']*100:.2f}%)")
    
    return results

# Train and evaluate classifiers
if all_data and 'feature_df' in dir() and len(feature_df) > 10:
    classification_results = compare_svm_kernels(feature_df)
else:
    print("Not enough data for classification. Need at least 10 samples.")

---

## Question 9 Response

**Compare the classification accuracy of Linear SVM vs. RBF SVM. Which performs better on your data, and why might this be?**

*Your observations:*

- Linear SVM accuracy: ___%
- RBF SVM accuracy: ___%
- The better performing kernel is: ___
- This might be because: ___

---

## Question 10 Response

**Examine the confusion matrix. Which gaze directions are most often confused with each other? Does this make sense given the electrode placement and expected signals?**

*Your observations:*

- Most commonly confused classes: ___ and ___
- This makes sense because: ___
- Classes that are well-separated: ___

---

### Effect of Reducing Gaze Classes

Let's see how classification accuracy changes with fewer classes:

In [ ]:
def test_reduced_classes(feature_df):
    """
    Test classification with different subsets of gaze classes.
    """
    class_subsets = [
        ['left', 'right'],                          # Horizontal only
        ['up', 'down'],                             # Vertical only
        ['left', 'right', 'center'],                # Horizontal + center
        ['left', 'right', 'up', 'down'],            # Cardinal directions
        ['left', 'right', 'up', 'down', 'center'],  # All directions
        ['left', 'right', 'up', 'down', 'center', 'blink'],  # + blink
    ]
    
    results = []
    
    for classes in class_subsets:
        # Filter to classes that exist in our data
        available_classes = [c for c in classes if c in feature_df['gaze'].unique()]
        
        if len(available_classes) < 2:
            continue
        
        try:
            X, y, _, label_encoder, _ = prepare_classification_data(
                feature_df, available_classes
            )
            
            if len(np.unique(y)) < 2:
                continue
            
            # Quick cross-validation with RBF SVM
            model = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
            cv_scores = cross_val_score(model, X, y, cv=min(5, len(y)//2), scoring='accuracy')
            
            results.append({
                'classes': available_classes,
                'n_classes': len(available_classes),
                'n_samples': len(y),
                'accuracy': cv_scores.mean(),
                'std': cv_scores.std()
            })
        except Exception as e:
            print(f"Error with classes {available_classes}: {e}")
    
    # Display results
    if results:
        print("\nClassification Accuracy by Number of Classes:")
        print("-" * 70)
        for r in results:
            class_str = ', '.join(r['classes'])
            print(f"  {r['n_classes']} classes ({r['n_samples']:3d} samples): "
                  f"{r['accuracy']*100:.1f}% +/- {r['std']*100:.1f}%")
            print(f"    Classes: {class_str}")
        
        # Plot
        fig, ax = plt.subplots(figsize=(10, 5))
        x = [r['n_classes'] for r in results]
        y = [r['accuracy'] * 100 for r in results]
        yerr = [r['std'] * 100 for r in results]
        
        ax.errorbar(x, y, yerr=yerr, fmt='o-', capsize=5, markersize=10)
        ax.set_xlabel('Number of Gaze Classes')
        ax.set_ylabel('Classification Accuracy (%)')
        ax.set_title('Classification Accuracy vs. Number of Classes')
        ax.set_xticks(x)
        ax.set_ylim(0, 105)
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
    
    return results

if all_data and 'feature_df' in dir() and len(feature_df) > 10:
    reduced_results = test_reduced_classes(feature_df)

---

## Question 11 Response

**How does reducing the number of gaze classes affect classification accuracy?**

*Your observations:*

- 2-class (left/right) accuracy: ___%
- 3-class accuracy: ___%
- 5-class accuracy: ___%
- The trend shows: ___
- For a practical interface, I would recommend ___ classes because: ___

---

## Part 7: Summary and Conclusions

### Key Findings

Summarize your main findings from this analysis:

In [ ]:
# Generate summary statistics
if all_data and 'feature_df' in dir() and 'classification_results' in dir():
    print("="*60)
    print("  LAB 2 ANALYSIS SUMMARY")
    print("="*60)
    
    # Data summary
    print("\n📊 DATA COLLECTED:")
    print(f"   Total trials: {len(feature_df)}")
    print(f"   Gaze classes: {list(feature_df['gaze'].unique())}")
    if 'participant' in feature_df.columns:
        print(f"   Participants: {list(feature_df['participant'].unique())}")
    
    # Feature summary
    print("\n📈 FEATURES:")
    exclude_cols = ['gaze', 'trial', 'participant']
    n_features = len([c for c in feature_df.columns if c not in exclude_cols])
    print(f"   Number of features: {n_features}")
    
    # PCA summary
    if 'pca' in dir():
        print("\n🔄 DIMENSIONALITY REDUCTION (PCA):")
        print(f"   PC1 variance: {pca.explained_variance_ratio_[0]*100:.1f}%")
        print(f"   PC2 variance: {pca.explained_variance_ratio_[1]*100:.1f}%")
    
    # Classification summary
    print("\n🎯 CLASSIFICATION RESULTS:")
    print(f"   Linear SVM: {classification_results['linear']['accuracy']*100:.1f}%")
    print(f"   RBF SVM: {classification_results['rbf']['accuracy']*100:.1f}%")
    
    best_kernel = 'rbf' if classification_results['rbf']['accuracy'] > classification_results['linear']['accuracy'] else 'linear'
    print(f"   Best kernel: {best_kernel.upper()}")
    
    print("\n" + "="*60)

---

## Discussion Questions

Answer these final questions for your lab report:

### Question 12
**What are the main differences between EOG and EMG signals in terms of frequency content, amplitude, and signal characteristics?**

*Your answer:*

---

### Question 13
**Why might EOG-based gaze tracking be preferred over camera-based eye tracking in certain applications? What are the limitations of EOG?**

*Your answer:*

---

### Question 14
**If you were designing a practical EOG-based interface (e.g., for a wheelchair or computer control), which gaze commands would you include and why? How would you handle false positives?**

*Your answer:*

---

### Question 15
**How could the blink signal be used as an additional control input rather than just an artifact to be removed?**

*Your answer:*

---

## Save Results

Save your processed data and results for the lab report:

In [ ]:
# Save feature dataframe
if 'feature_df' in dir():
    output_path = DATA_DIR / 'features_extracted.csv'
    feature_df.to_csv(output_path, index=False)
    print(f"Features saved to: {output_path}")

# Save classification results
if 'classification_results' in dir():
    results_summary = {
        'linear_accuracy': classification_results['linear']['accuracy'],
        'linear_cv_mean': classification_results['linear']['cv_mean'],
        'rbf_accuracy': classification_results['rbf']['accuracy'],
        'rbf_cv_mean': classification_results['rbf']['cv_mean'],
    }
    results_df = pd.DataFrame([results_summary])
    results_path = DATA_DIR / 'classification_results.csv'
    results_df.to_csv(results_path, index=False)
    print(f"Classification results saved to: {results_path}")

---

## Appendix: Synthetic Data for Testing

If you don't have real data yet, you can generate synthetic EOG data to test the analysis pipeline:

In [ ]:
def generate_synthetic_eog(gaze_direction, duration_s=3.0, sample_rate=250):
    """
    Generate synthetic EOG data for testing.
    
    Args:
        gaze_direction: 'center', 'left', 'right', 'up', 'down', 'blink'
        duration_s: duration in seconds
        sample_rate: samples per second
        
    Returns:
        DataFrame with heog, veog columns
    """
    n_samples = int(duration_s * sample_rate)
    time = np.arange(n_samples) / sample_rate
    
    # Base noise
    noise_level = 10  # μV
    heog = np.random.randn(n_samples) * noise_level
    veog = np.random.randn(n_samples) * noise_level
    
    # Add gaze-specific signals
    amplitude = 100  # μV for full gaze shift
    
    if gaze_direction == 'left':
        heog += -amplitude + np.random.randn() * 20
    elif gaze_direction == 'right':
        heog += amplitude + np.random.randn() * 20
    elif gaze_direction == 'up':
        veog += amplitude + np.random.randn() * 20
    elif gaze_direction == 'down':
        veog += -amplitude + np.random.randn() * 20
    elif gaze_direction == 'blink':
        # Add blink artifact at random position
        blink_pos = np.random.randint(n_samples // 4, 3 * n_samples // 4)
        blink_width = int(0.15 * sample_rate)  # 150ms blink
        blink_shape = np.exp(-((np.arange(blink_width) - blink_width//2)**2) / (2 * (blink_width/4)**2))
        veog[blink_pos:blink_pos+blink_width] += 300 * blink_shape
    elif gaze_direction == 'double_blink':
        # Add two blink artifacts
        for offset in [0.3, 0.6]:
            blink_pos = int(offset * n_samples)
            blink_width = int(0.15 * sample_rate)
            blink_shape = np.exp(-((np.arange(blink_width) - blink_width//2)**2) / (2 * (blink_width/4)**2))
            if blink_pos + blink_width < n_samples:
                veog[blink_pos:blink_pos+blink_width] += 300 * blink_shape
    # center: just noise
    
    return pd.DataFrame({
        'timestamp': time,
        'heog': heog,
        'veog': veog
    })


def generate_synthetic_dataset(trials_per_class=15, gaze_classes=GAZE_CLASSES):
    """
    Generate a complete synthetic dataset.
    """
    data = {gaze: [] for gaze in gaze_classes}
    
    for gaze in gaze_classes:
        for trial in range(trials_per_class):
            df = generate_synthetic_eog(gaze)
            df['participant'] = 'synthetic'
            df['trial'] = trial
            df['gaze'] = gaze
            data[gaze].append(df)
    
    return data

# Generate synthetic data if no real data available
if all_data is None or all([len(v) == 0 for v in all_data.values()]):
    print("Generating synthetic EOG data for demonstration...")
    all_data = generate_synthetic_dataset(trials_per_class=15)
    print("Synthetic data generated!")
    
    # Extract features from synthetic data
    feature_df = extract_all_features(all_data)
    print(f"Extracted features from {len(feature_df)} synthetic trials")

---

*End of Lab 2 Analysis Notebook*